In [3]:
import requests
from bs4 import BeautifulSoup


def scrapetle():
    url = 'http://celestrak.com/NORAD/elements/stations.txt'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    mytle=(soup.text[:168])
    lines = mytle.splitlines()
    return (lines)

In [4]:
import math
import time
from datetime import datetime, timedelta
import ephem
import numpy as np
 

degrees_per_radian = 180.0 / math.pi
 
home = ephem.Observer()
home.lon = '0'   # +E
home.lat = '0'      # +N
home.elevation = 50 # meters
 
# Always get the latest ISS TLE data from:
lines=scrapetle()
# http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
iss = ephem.readtle(lines[0], lines[1], lines[2])



a=(2016, 4, 7, 9, 10, 45)
a=(2016, 4, 8, 21, 10, 45)

mydatenow=datetime.utcnow()


def getantimeridancrossingtime(time):
    import math
    import time
    from datetime import datetime, timedelta
    import ephem
    import numpy as np
    #mydate=datetime(2016, 4, 7, 9, 10, 45)
    mydatenow=datetime.utcnow()

    home.date=mydatenow
    iss.compute(home)

    issorbitsperday=15.54459501994820
    issorbitalperiod=24*60/issorbitsperday


    print ("ISS sublong" , iss.sublong)
    sublong=iss.sublong.real

    print ("Position in degrees", sublong*180/np.pi)

    radtodeg=180/np.pi

    angletoam=(sublong+np.pi)
    angletoamdeg=angletoam*radtodeg
    print ("Angle to AM in degrees", angletoam*radtodeg)
    timetoam=(angletoamdeg)*issorbitalperiod/360


    deltatoam=timedelta(minutes=timetoam)
    mydate=mydatenow-deltatoam


    print ("Minutes to antimeridian" , timetoam)

    print ("time Delta to antimeridian" , deltatoam)
    print ("Now ", mydatenow)

    print ("Antimeridian time", mydate)
    home.date = mydate
    iss.compute(home)
            
    print (iss.sublong)
    return (mydate)

mydate=getantimeridancrossingtime(time)
#dir(sublong)


ISS sublong 167:57:13.1
Position in degrees 167.95364135093988
Angle to AM in degrees 347.95364135093985
Minutes to antimeridian 89.53688170181724
time Delta to antimeridian 1:29:32.212902
Now  2016-05-05 15:33:58.524159
Antimeridian time 2016-05-05 14:04:26.311257
-161:55:59.4


In [8]:
#datearr=np.array(['2016-04-07T09:10', '2002-04-10T09:10:03.172'], dtype='datetime64')
npoints=18

def getorbit(mydate,npoints):
    lonarr=[0] * npoints
    latarr=[0] * npoints
#for q in range (0,2):
    for i in range(0,npoints):
        one_hour = timedelta(hours=1)
        min15 = timedelta(minutes=5)
        #mydate=datearr[q]
        #mydate=datetime(a[0], a[1], a[2], a[3], a[4], a[5])
        home.date = mydate +i*min15
        iss.compute(home)
        dist=2
        lonarr[i]=iss.sublong* degrees_per_radian
        latarr[i]=iss.sublat* degrees_per_radian
        #if ( (iss.alt < dist) and (iss.az < dist)):
        #    print (home.date)
        #    print('iss: altitude %4.1f deg, azimuth %5.1f deg' % (iss.alt * degrees_per_radian, iss.az * degrees_per_radian))
 
    print (lonarr)
    print (latarr)
    return (lonarr, latarr)


lonarr, latarr=getorbit(mydate, npoints)

[-161.93317110703214, -150.61132659541394, -136.95108485917183, -118.31341106717463, -92.41452901759104, -62.80878719222562, -38.143065061436566, -20.562308401985483, -7.463088158884473, 3.668967166927049, 14.725433642967685, 27.565893876910053, 44.62340852706667, 68.56876085137824, 97.93349262169059, 124.34689965164041, 143.53958095521452, 157.51993999747677]
[3.161666392268576, 18.208311025297267, 32.29110241454221, 44.02262508767235, 50.90941262024241, 50.21876779703282, 42.28765117055698, 30.015887854835505, 15.694212306916661, 0.5732406482053922, -14.563737452104096, -28.95313091054903, -41.41605756076082, -49.79176144559655, -51.18138392280315, -44.91696322740465, -33.55678086397518, -19.662161186364088]


In [7]:
import gpxpy 
import gpxpy.gpx 

gpx = gpxpy.gpx.GPX() 

# Create first track in our GPX: 
gpx_track = gpxpy.gpx.GPXTrack() 
gpx.tracks.append(gpx_track) 

# Create first segment in our GPX track: 
gpx_segment = gpxpy.gpx.GPXTrackSegment() 
gpx_track.segments.append(gpx_segment) 

# Create points: 

for i in range(0,npoints):
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(latarr[i], lonarr[i], elevation=1234)) 
#    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(2.1236, 5.1236, elevation=1236)) 

# You can add routes and waypoints, too... 

print ('Created GPX:', gpx.to_xml() )


Created GPX: <?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.topografix.com/GPX/1/0" xsi:schemaLocation="http://www.topografix.com/GPX/1/0 http://www.topografix.com/GPX/1/0/gpx.xsd" version="1.0" creator="gpx.py -- https://github.com/tkrajina/gpxpy">
<trk>
<trkseg>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat="3.161666392268576" lon="-161.93317110703214">
<ele>1234</ele></trkpt>
<trkpt lat